Alohan'ny mamerina dia avereno atao Run ny notebook iray manontolo. Ny fanaovana azy dia redémarrena mihitsy ny kernel aloha (jereo menubar, safidio **Kernel$\rightarrow$Restart Kernel and Run All Cells**).

Izay misy hoe `YOUR CODE HERE` na "YOUR ANSWER HERE" ihany no fenoina. Afaka manampy cells vaovao raha ilaina. Aza adino ny mameno references eo ambany raha ilaina.

## References
Eto ilay references rehetra no apetraka

# https://www.cvxpy.org/examples/basic/least_squares.html
# https://www.cvxpy.org/examples/machine_learning/ridge_regression.html

# Linear regression

In [1]:
import numpy as np
import scipy
from sklearn.metrics import mean_squared_error
from sklearn import datasets
from sklearn.linear_model import LinearRegression as LR
import cvxpy as cp
from sklearn.linear_model import Ridge, Lasso
import warnings
warnings.filterwarnings("ignore")

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

def generate_data(N=100, d=20, sigma=5):
    """ Data for Ridge """
    np.random.seed(1)
    w_star = np.random.randn(d)
    X = np.random.randn(N, d)
    y = X.dot(w_star) + np.random.normal(0, sigma, size=N)
    return X, y

def generate_data_lasso(N=100, d=20, sigma=5, density=0.2):
    """ Data for Lasso """
    np.random.seed(1)
    w_star = np.random.randn(d)
    idxs = np.random.choice(range(d), int((1-density)*d), replace=False)
    for idx in idxs:
        w_star[idx] = 0
    X = np.random.randn(N,d)
    y = X.dot(w_star) + np.random.normal(0, sigma, size=N)
    return X, y

def sigmoid(z):
    return 1/(1 + np.exp(-z))

def generate_data_log_reg(N=50, d=50):
    np.random.seed(1)
    w_star = np.array([1, 0.5, -0.5] + [0]*(d - 3))
    X = (np.random.random((N, d)) - 0.5)*10
    y = np.round(sigmoid(X @ w_star + np.random.randn(N)*0.5))
    return X, y

data = datasets.load_diabetes()
X_train, y_train = data.data, data.target
X_train2, y_train2 = generate_data()
X_train3, y_train3 = generate_data_lasso()
X_train4, y_train4 = generate_data_log_reg()

In [2]:
class LinearRegression():
    def __init__(self, fit_intercept=True):
        self.w = 0
        self.fit_intercept = fit_intercept # bias
    
    def fit(self, X, y):
        # YOUR CODE HERE
        if self.fit_intercept:
            X = np.concatenate((np.ones((X.shape[0],1)), X),axis=1)
        x = cp.Variable(X.shape[1])
        problem = cp.sum_squares(X @ x - y)
        prob = cp.Problem(cp.Minimize(problem))
        prob.solve()
        self.w = x.value
        
    def predict(self, X):
        # YOUR CODE HERE
        if self.fit_intercept == True:
            X = np.concatenate((np.ones((X.shape[0],1)), X),axis=1)
        return X.dot(self.w)

In [3]:
# Without bias
sk_model = LR(fit_intercept=False)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model = LinearRegression(fit_intercept=False)
model.fit(X_train, y_train)
pred = model.predict(X_train)

error = rel_error(pred, sk_pred)
print("prediction error: ", error)
assert error <= 1e-11

prediction error:  7.905987949911426e-14


In [4]:
# With bias
sk_model = LR(fit_intercept=True)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model = LinearRegression(fit_intercept=True)
model.fit(X_train, y_train)
pred = model.predict(X_train)

error = rel_error(pred, sk_pred)
print("prediction error: ", error)
assert error <= 1e-11

prediction error:  1.1190861660841687e-15


# Ridge regression

In [5]:
class RidgeRegression():
    def __init__(self, fit_intercept=True, alpha=1.0):
        self.w = 0
        self.fit_intercept = fit_intercept # bias
        self.alpha = alpha
    
    def fit(self, X, y):
        # YOUR CODE HERE
        if self.fit_intercept:
            X = np.concatenate((np.ones((X.shape[0],1)), X),axis=1)
        x = cp.Variable(X.shape[1])
        problem = cp.sum_squares(X @ x - y) + (self.alpha * (cp.pnorm(x, p=2)**2))

        prob = cp.Problem(cp.Minimize(problem))
        prob.solve()
        self.w = x.value
               
    def predict(self, X):
        # YOUR CODE HERE
        if self.fit_intercept == True:
            X = np.concatenate((np.ones((X.shape[0],1)), X),axis=1)
        return X.dot(self.w)

In [6]:
# without bias
model = RidgeRegression(fit_intercept=False, alpha=0.1)
model.fit(X_train2, y_train2)

w_solution = [2.2741331962708733,-1.4638470967067754,-1.0248494680125682,-2.0920403465511344,0.19793283915844787,-1.5186692704860287,1.4772054728555917,-0.5873242037184364,0.9478891631775056,0.20512816292505345,1.251288772139991,-2.681990788073989,0.04476204682607866,-0.8659943546608414,0.6794151132231774,-0.45806886087608134,-0.11772977214105436,-1.5167038016358336,-0.7285498050097046,1.1970655855063765]
error = rel_error(model.w, w_solution)
print("prediction error: ", error)
assert error <= 1e-8

prediction error:  3.0908374407670053e-06


AssertionError: 

In [7]:
# with bias
model = RidgeRegression(fit_intercept=True, alpha=0.1)
model.fit(X_train2, y_train2)

w_solution = [-0.12421153467148652, 2.2885621086080183, -1.460016084362311, -1.0386230518778734, -2.0755554006911163, 0.16722384639912463, -1.5196366460908797, 1.490644600189988, -0.5506589908428944, 0.953560073286487, 0.20519345577354192, 1.2565834667864626, -2.6559028064874886, 0.05943949693736531, -0.8413627640000328, 0.689138089040695, -0.4717409588520616, -0.11380803855096185, -1.5157445906226719, -0.7155151711254747, 1.2094429722709097]
error = rel_error(model.w, w_solution)
print("prediction error: ", error)
assert error <= 1e-8

prediction error:  0.0006379157053825171


AssertionError: 

# Lasso

In [8]:
class LassoRegression():
    def __init__(self, fit_intercept=True, alpha=1.0):
        self.w = 0
        self.fit_intercept = fit_intercept # bias
        self.alpha = alpha
    
    def fit(self, X, y):
        # YOUR CODE HERE
        if self.fit_intercept:
            X = np.concatenate((np.ones((X.shape[0],1)), X),axis=1)
        x = cp.Variable(X.shape[1])
        problem = cp.sum_squares(X @ x - y) + (self.alpha * cp.norm1(x))
        prob = cp.Problem(cp.Minimize(problem))
        prob.solve()
        self.w = x.value
        
    def predict(self, X):
        # YOUR CODE HERE
        if self.fit_intercept == True:
            X = np.concatenate((np.ones((X.shape[0],1)), X),axis=1)
        return X.dot(self.w)

In [9]:
# without bias
model = LassoRegression(fit_intercept=False, alpha=0.1)
model.fit(X_train3, y_train3)

w_solution = [-0.8521262649671281, 0.0254836890059677, 0.7137682249492029, -0.8784736131759308, 0.26239208158878835, 0.6462086038227195, 0.6430994649127592, -0.6427109854827273, 0.8457229064959301, -0.3402411535357167, 0.33481565380057277, -2.2109314288098636, 0.22685158332884042, -0.9969899386988903, -0.486582184431374, -0.0654138939227482, 0.5269406964201837, -1.2991221762643268, -0.1472881184306273, -0.749580456217885]
error = rel_error(model.w, w_solution)
print("prediction error: ", error)
assert error <= 1e-3

prediction error:  0.00016772874776678658


In [10]:
# with bias
model = LassoRegression(fit_intercept=True, alpha=0.1)
model.fit(X_train3, y_train3)

w_solution = [0.02773625477624174, -0.8487201678165991, 0.025083418700813768, 0.7190663030638081, -0.8749520697657432, 0.26342660512442967, 0.6431043993589891, 0.6494385810352119, -0.6408605760028508, 0.8380835575881868, -0.34441398162201164, 0.3295390950817965, -2.2106797474729007, 0.22549557495145922, -0.9948431209736727, -0.48738779274910293, -0.06222135539355891, 0.52148327011433, -1.3019085858141572, -0.15286852250292499, -0.7487483354438857]
error = rel_error(model.w, w_solution)
print("prediction error: ", error)
assert error <= 1e-3

prediction error:  0.012490139506555793


AssertionError: 